In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [3]:
#Prompt
MAIN_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow of a use case.
Then you predict all scenarios that can happen in this flow.

Rules to predict scenarios:
- Stay close to the details described in the flow.
- Choose important cases to generate, important scenario is the scenario that users are more likely to encounter it.
- Limit the appearance of scenarios that are hard to happen. 
- A scenario encompasses a whole function, not just verifying individual steps.
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

SUB_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow and one alternative or exception flow of a use case.
Then you predict all scenarios that can lead user from the main flow to change to the alternative or exception flow mentioned for creating test cases.

Rules to predict scenarios:
- A scenario encompasses a whole function, not just verifying individual steps.
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- Do not choose another option that is not chosen by the flow, eventhough it is mentioned (Example: A pop up with OK and Cancel, the flow only has step choose OK. Do not generate scenario that press Cancel)
- Do not generate scenario to test only the main flow.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

EXTRACT_CONDITION_SYSTEM_PROMPT="""
Given use case flows of a feature.
Your task is to identify all the interactive elements within the feature. 
For each interactive element:
Identify what type of that element (button,buttons, icon,scroll, text field,text area, tab, radio buttons, menu, combobox, sliders, switches, dialog, link, form,rating, filter).
Identify all the conditions mentioned in the use case of that element that would make the element valid and the conditions that would make the element invalid based on the description of the use case flow.
Do not arbitrarily create additional conditions that not mention in the use case flow.
Return the element extracted in json format.
The JSON format should follow the following structure:
{"Name of interactive element": {"condition": {valid:"conditions that make element valid", invalid: "conditions that make element invalid"}, "type": "element type"}}
Examples of output json format template: 
{"Username": {"condition": {"valid": "must be over 8 characters and below 30 characters, must be entered", racter, empty"}, "type": "text field"}}
{"Search button": {"condition": {"valid": "", "invalid": ""}, "type": "text field"}}
"""

GEN_SCENARIO_FOR_CONDITION_PROMPT="""
Given a list of interaction element for input value, their extracted conditions and corresponding use case.
For each given invalid condition:
- Generate a test scenario that test that condition.
Do not generate test scenario to test element/condition that not mention in the given element list.
Do not generate test scenario to test valid conditions.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""


FILTER_SYSTEM_PROMPT="""
Your task is to read all of this scenarios generation from many sources.
Then you remove all the same copies because there are many duplicate scenarios in meanings inside the input.
Make sure every scenarios in the response is unique.
Return scenario name only.
"""

# Identify what type of that element (button,buttons, icon,scroll, text field, radio buttons, menu, menu for navigation,menu for opening dialog or another menu, menu for filter, sliders, switches, dialog, link, form,rating, filter).


In [4]:
usecase_name = "AddQuestion"
project_name = "MatchaEnglishWebsite"
main_flow_prompt="""
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 3: Admin presses one Lesson on the Lesson list.
Step 4: The system redirects Admin to the "Lesson Information" page of selected lesson which show the information of that Lesson and Topic list included in that Lesson.
Step 5: Admin presses one Topic on the Topic list of selected Lesson.
Step 6: The system redirects Admin to the "Topic Information" page of selected topic which show the information of that selected topic and vocabulary word list included in that Topic.
Step 7: Admin presses one word on vocabulary word list.
Step 8: The system redirects Admin to the "Word Information" page of selected word which show the information of that selected word and button "Add new question".
Step 9: Admin presses button "Add new question".
Step 10: The system pop up dialog to choose type of question Admin want to create.
Step 11: Admin presses option add "Create questions for the selection test".
Step 12: The system redirects Admin to the "Add new question for selection test" page of selected word which show all the fields that admin need to input to create a selection test and button 'Save' and 'Cancel'. These complusary fields are: one textfield "sentence" for Admins to input sentence that learner need to fill in blank and the position that the word learner need to fill must present by symbol '...',. One textfield 'correct answer' for Admins to input the correct answer word, three textfield 'other option 1', 'other option 2', 'other option 3' for Admins to input the 3 different wrong answer word.
Step 13: Admin fills in the sentence field by a valid sentence has a symbol '...', a valid sentence must be below 250 characters and must not contains any special character. 
Step 14: Admin fills 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 15: Admin presses button 'Save' to save the question information.
Step 16: System save the question and redirect Admin to the "Word Information" page of selected word.
"""

alt_prompt_1="""
Main flow: 
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 3: Admin presses one Lesson on the Lesson list.
Step 4: The system redirects Admin to the "Lesson Information" page of selected lesson which show the information of that Lesson and Topic list included in that Lesson.
Step 5: Admin presses one Topic on the Topic list of selected Lesson.
Step 6: The system redirects Admin to the "Topic Information" page of selected topic which show the information of that selected topic and vocabulary word list included in that Topic.
Step 7: Admin presses one word on vocabulary word list.
Step 8: The system redirects Admin to the "Word Information" page of selected word which show the information of that selected word and button "Add new question".
Step 9: Admin presses button "Add new question".
Step 10: The system pop up dialog to choose type of question Admin want to create.
Step 11: Admin presses option add "Create questions for the selection test".
Step 12: The system redirects Admin to the "Add new question for selection test" page of selected word which show all the fields that admin need to input to create a selection test and button 'Save' and 'Cancel'. These complusary fields are: one textfield "sentence" for Admins to input sentence that learner need to fill in blank and the position that the word learner need to fill must present by symbol '...',. One textfield 'correct answer' for Admins to input the correct answer word, three textfield 'other option 1', 'other option 2', 'other option 3' for Admins to input the 3 different wrong answer word.
Step 13: Admin fills in the sentence field by a valid sentence has a symbol '...', a valid sentence must be below 250 characters and must not contains any special character. 
Step 14: Admin fills 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 15: Admin presses button 'Save' to save the question information.
Step 16: System save the question and redirect Admin to the "Word Information" page of selected word.

Alternative flow 1: Admin go to "Lesson Management" page from Homepage.
At step 1 of the basic flow: Admin presses the "Lesson Management" tab at Homepage.
Go back to step 2 in the basic flow and continue with the steps from step 2.
"""

alt_prompt_2="""
Main flow: 
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 3: Admin presses one Lesson on the Lesson list.
Step 4: The system redirects Admin to the "Lesson Information" page of selected lesson which show the information of that Lesson and Topic list included in that Lesson.
Step 5: Admin presses one Topic on the Topic list of selected Lesson.
Step 6: The system redirects Admin to the "Topic Information" page of selected topic which show the information of that selected topic and vocabulary word list included in that Topic.
Step 7: Admin presses one word on vocabulary word list.
Step 8: The system redirects Admin to the "Word Information" page of selected word which show the information of that selected word and button "Add new question".
Step 9: Admin presses button "Add new question".
Step 10: The system pop up dialog to choose type of question Admin want to create.
Step 11: Admin presses option add "Create questions for the selection test".
Step 12: The system redirects Admin to the "Add new question for selection test" page of selected word which show all the fields that admin need to input to create a selection test and button 'Save' and 'Cancel'. These complusary fields are: one textfield "sentence" for Admins to input sentence that learner need to fill in blank and the position that the word learner need to fill must present by symbol '...',. One textfield 'correct answer' for Admins to input the correct answer word, three textfield 'other option 1', 'other option 2', 'other option 3' for Admins to input the 3 different wrong answer word.
Step 13: Admin fills in the sentence field by a valid sentence has a symbol '...', a valid sentence must be below 250 characters and must not contains any special character. 
Step 14: Admin fills 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 15: Admin presses button 'Save' to save the question information.
Step 16: System save the question and redirect Admin to the "Word Information" page of selected word.

Alternative flow 2: Admin skip complusary field "sentence".
At step 13 of the basic flow: Admin field all the complusary field except field "sentence".
Step 14: Admin fills 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 15: Admin presses button 'Save'.
Step 16: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing "sentence" field.
Step 17: Admin re-fill the missing field.
Step 18: Admin presses button 'Save'.
"""

alt_prompt_3="""
Main flow: 
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 3: Admin presses one Lesson on the Lesson list.
Step 4: The system redirects Admin to the "Lesson Information" page of selected lesson which show the information of that Lesson and Topic list included in that Lesson.
Step 5: Admin presses one Topic on the Topic list of selected Lesson.
Step 6: The system redirects Admin to the "Topic Information" page of selected topic which show the information of that selected topic and vocabulary word list included in that Topic.
Step 7: Admin presses one word on vocabulary word list.
Step 8: The system redirects Admin to the "Word Information" page of selected word which show the information of that selected word and button "Add new question".
Step 9: Admin presses button "Add new question".
Step 10: The system pop up dialog to choose type of question Admin want to create.
Step 11: Admin presses option add "Create questions for the selection test".
Step 12: The system redirects Admin to the "Add new question for selection test" page of selected word which show all the fields that admin need to input to create a selection test and button 'Save' and 'Cancel'. These complusary fields are: one textfield "sentence" for Admins to input sentence that learner need to fill in blank and the position that the word learner need to fill must present by symbol '...',. One textfield 'correct answer' for Admins to input the correct answer word, three textfield 'other option 1', 'other option 2', 'other option 3' for Admins to input the 3 different wrong answer word.
Step 13: Admin fills in the sentence field by a valid sentence has a symbol '...', a valid sentence must be below 250 characters and must not contains any special character. 
Step 14: Admin fills 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 15: Admin presses button 'Save' to save the question information.
Step 16: System save the question and redirect Admin to the "Word Information" page of selected word.

Alternative flow 3: Admin skip complusary field "'correct answer'.
At step 14 of the basic flow: Admin field all the complusary field except field 'correct answer'.
Step 15: Admin presses button 'Save'.
Step 16: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'correct answer' field.
Step 17: Admin re-fill the missing field.
Step 18: Admin presses button 'Save'.
"""
alt_prompt_4="""
Main flow: 
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 3: Admin presses one Lesson on the Lesson list.
Step 4: The system redirects Admin to the "Lesson Information" page of selected lesson which show the information of that Lesson and Topic list included in that Lesson.
Step 5: Admin presses one Topic on the Topic list of selected Lesson.
Step 6: The system redirects Admin to the "Topic Information" page of selected topic which show the information of that selected topic and vocabulary word list included in that Topic.
Step 7: Admin presses one word on vocabulary word list.
Step 8: The system redirects Admin to the "Word Information" page of selected word which show the information of that selected word and button "Add new question".
Step 9: Admin presses button "Add new question".
Step 10: The system pop up dialog to choose type of question Admin want to create.
Step 11: Admin presses option add "Create questions for the selection test".
Step 12: The system redirects Admin to the "Add new question for selection test" page of selected word which show all the fields that admin need to input to create a selection test and button 'Save' and 'Cancel'. These complusary fields are: one textfield "sentence" for Admins to input sentence that learner need to fill in blank and the position that the word learner need to fill must present by symbol '...',. One textfield 'correct answer' for Admins to input the correct answer word, three textfield 'other option 1', 'other option 2', 'other option 3' for Admins to input the 3 different wrong answer word.
Step 13: Admin fills in the sentence field by a valid sentence has a symbol '...', a valid sentence must be below 250 characters and must not contains any special character. 
Step 14: Admin fills 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 15: Admin presses button 'Save' to save the question information.
Step 16: System save the question and redirect Admin to the "Word Information" page of selected word.

Alternative flow 4: Admin skip complusary field 'other option 1'.
At step 14 of the basic flow: Admin field all the complusary field except field 'other option 1'.
Step 15: Admin presses button 'Save'.
Step 16: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'other option 1' field.
Step 17: Admin re-fill the missing field.
Step 18: Admin presses button 'Save'.
"""

alt_prompt_5="""
Main flow: 
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 3: Admin presses one Lesson on the Lesson list.
Step 4: The system redirects Admin to the "Lesson Information" page of selected lesson which show the information of that Lesson and Topic list included in that Lesson.
Step 5: Admin presses one Topic on the Topic list of selected Lesson.
Step 6: The system redirects Admin to the "Topic Information" page of selected topic which show the information of that selected topic and vocabulary word list included in that Topic.
Step 7: Admin presses one word on vocabulary word list.
Step 8: The system redirects Admin to the "Word Information" page of selected word which show the information of that selected word and button "Add new question".
Step 9: Admin presses button "Add new question".
Step 10: The system pop up dialog to choose type of question Admin want to create.
Step 11: Admin presses option add "Create questions for the selection test".
Step 12: The system redirects Admin to the "Add new question for selection test" page of selected word which show all the fields that admin need to input to create a selection test and button 'Save' and 'Cancel'. These complusary fields are: one textfield "sentence" for Admins to input sentence that learner need to fill in blank and the position that the word learner need to fill must present by symbol '...',. One textfield 'correct answer' for Admins to input the correct answer word, three textfield 'other option 1', 'other option 2', 'other option 3' for Admins to input the 3 different wrong answer word.
Step 13: Admin fills in the sentence field by a valid sentence has a symbol '...', a valid sentence must be below 250 characters and must not contains any special character. 
Step 14: Admin fills 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 15: Admin presses button 'Save' to save the question information.
Step 16: System save the question and redirect Admin to the "Word Information" page of selected word.

Alternative flow 5: Admin skip complusary field 'other option 2'.
At step 14 of the basic flow: Admin field all the complusary field except field 'other option 2'.
Step 15: Admin presses button 'Save'.
Step 16: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'other option 2' field.
Step 17: Admin re-fill the missing field.
Step 18: Admin presses button 'Save'.
"""

alt_prompt_6="""
Main flow: 
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 3: Admin presses one Lesson on the Lesson list.
Step 4: The system redirects Admin to the "Lesson Information" page of selected lesson which show the information of that Lesson and Topic list included in that Lesson.
Step 5: Admin presses one Topic on the Topic list of selected Lesson.
Step 6: The system redirects Admin to the "Topic Information" page of selected topic which show the information of that selected topic and vocabulary word list included in that Topic.
Step 7: Admin presses one word on vocabulary word list.
Step 8: The system redirects Admin to the "Word Information" page of selected word which show the information of that selected word and button "Add new question".
Step 9: Admin presses button "Add new question".
Step 10: The system pop up dialog to choose type of question Admin want to create.
Step 11: Admin presses option add "Create questions for the selection test".
Step 12: The system redirects Admin to the "Add new question for selection test" page of selected word which show all the fields that admin need to input to create a selection test and button 'Save' and 'Cancel'. These complusary fields are: one textfield "sentence" for Admins to input sentence that learner need to fill in blank and the position that the word learner need to fill must present by symbol '...',. One textfield 'correct answer' for Admins to input the correct answer word, three textfield 'other option 1', 'other option 2', 'other option 3' for Admins to input the 3 different wrong answer word.
Step 13: Admin fills in the sentence field by a valid sentence has a symbol '...', a valid sentence must be below 250 characters and must not contains any special character. 
Step 14: Admin fills 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 15: Admin presses button 'Save' to save the question information.
Step 16: System save the question and redirect Admin to the "Word Information" page of selected word.

Alternative flow 6: Admin skip complusary field 'other option 3'.
At step 14 of the basic flow: Admin field all the complusary field except field 'other option 3'.
Step 15: Admin presses button 'Save'.
Step 16: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'other option 3'field.
Step 17: Admin re-fill the missing field.
Step 18: Admin presses button 'Save'.
"""

alt_prompt_7="""
Main flow: 
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 3: Admin presses one Lesson on the Lesson list.
Step 4: The system redirects Admin to the "Lesson Information" page of selected lesson which show the information of that Lesson and Topic list included in that Lesson.
Step 5: Admin presses one Topic on the Topic list of selected Lesson.
Step 6: The system redirects Admin to the "Topic Information" page of selected topic which show the information of that selected topic and vocabulary word list included in that Topic.
Step 7: Admin presses one word on vocabulary word list.
Step 8: The system redirects Admin to the "Word Information" page of selected word which show the information of that selected word and button "Add new question".
Step 9: Admin presses button "Add new question".
Step 10: The system pop up dialog to choose type of question Admin want to create.
Step 11: Admin presses option add "Create questions for the selection test".
Step 12: The system redirects Admin to the "Add new question for selection test" page of selected word which show all the fields that admin need to input to create a selection test and button 'Save' and 'Cancel'. These complusary fields are: one textfield "sentence" for Admins to input sentence that learner need to fill in blank and the position that the word learner need to fill must present by symbol '...',. One textfield 'correct answer' for Admins to input the correct answer word, three textfield 'other option 1', 'other option 2', 'other option 3' for Admins to input the 3 different wrong answer word.
Step 13: Admin fills in the sentence field by a valid sentence has a symbol '...', a valid sentence must be below 250 characters and must not contains any special character. 
Step 14: Admin fills 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 15: Admin presses button 'Save' to save the question information.
Step 16: System save the question and redirect Admin to the "Word Information" page of selected word.

Alternative flow 7: Admin choose cancel the question and choose to stay.
At step 13 of the basic flow: Admin presses button 'Cancel'.
Step 14: System show dialog 'Are you sure you want to cancel the question?' with 2 option buttons 'Confirm' and 'Stay'.
Step 15: Admin presses the buttons 'Stay'.
Step 16: The dialog closed and the work on that question was kept.
Step 17: Admin presses button 'Save'.
"""

alt_prompt_8="""
Main flow: 
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 3: Admin presses one Lesson on the Lesson list.
Step 4: The system redirects Admin to the "Lesson Information" page of selected lesson which show the information of that Lesson and Topic list included in that Lesson.
Step 5: Admin presses one Topic on the Topic list of selected Lesson.
Step 6: The system redirects Admin to the "Topic Information" page of selected topic which show the information of that selected topic and vocabulary word list included in that Topic.
Step 7: Admin presses one word on vocabulary word list.
Step 8: The system redirects Admin to the "Word Information" page of selected word which show the information of that selected word and button "Add new question".
Step 9: Admin presses button "Add new question".
Step 10: The system pop up dialog to choose type of question Admin want to create.
Step 11: Admin presses option add "Create questions for the selection test".
Step 12: The system redirects Admin to the "Add new question for selection test" page of selected word which show all the fields that admin need to input to create a selection test and button 'Save' and 'Cancel'. These complusary fields are: one textfield "sentence" for Admins to input sentence that learner need to fill in blank and the position that the word learner need to fill must present by symbol '...',. One textfield 'correct answer' for Admins to input the correct answer word, three textfield 'other option 1', 'other option 2', 'other option 3' for Admins to input the 3 different wrong answer word.
Step 13: Admin fills in the sentence field by a valid sentence has a symbol '...', a valid sentence must be below 250 characters and must not contains any special character. 
Step 14: Admin fills 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 15: Admin presses button 'Save' to save the question information.
Step 16: System save the question and redirect Admin to the "Word Information" page of selected word.

Alternative flow 8: Admin enters invalid sentences.
At step 13 of the basic flow: Admin fills in the 'sentence' field by an invalid sentence.
Step 14: System show notification that 'Sentence is invalid', prompt Admin to re-enter the "sentence" field.
Step 15: Admin fix the sentence field by a valid sentence.
Step 16: Admin fills 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 17: Admin presses button 'Save'.
"""

exc_prompt_1="""
Main flow: 
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 3: Admin presses one Lesson on the Lesson list.
Step 4: The system redirects Admin to the "Lesson Information" page of selected lesson which show the information of that Lesson and Topic list included in that Lesson.
Step 5: Admin presses one Topic on the Topic list of selected Lesson.
Step 6: The system redirects Admin to the "Topic Information" page of selected topic which show the information of that selected topic and vocabulary word list included in that Topic.
Step 7: Admin presses one word on vocabulary word list.
Step 8: The system redirects Admin to the "Word Information" page of selected word which show the information of that selected word and button "Add new question".
Step 9: Admin presses button "Add new question".
Step 10: The system pop up dialog to choose type of question Admin want to create.
Step 11: Admin presses option add "Create questions for the selection test".
Step 12: The system redirects Admin to the "Add new question for selection test" page of selected word which show all the fields that admin need to input to create a selection test and button 'Save' and 'Cancel'. These complusary fields are: one textfield "sentence" for Admins to input sentence that learner need to fill in blank and the position that the word learner need to fill must present by symbol '...',. One textfield 'correct answer' for Admins to input the correct answer word, three textfield 'other option 1', 'other option 2', 'other option 3' for Admins to input the 3 different wrong answer word.
Step 13: Admin fills in the sentence field by a valid sentence has a symbol '...', a valid sentence must be below 250 characters and must not contains any special character. 
Step 14: Admin fills 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 15: Admin presses button 'Save' to save the question information.
Step 16: System save the question and redirect Admin to the "Word Information" page of selected word.

Exception flow 1: Admin leave by button back on browser without saving the question.
At step 13 of the basic flow: Admin presses the button back on the browser. 
Step 14: The system redirects Admin to the "Word Information" page of selected word and Question is not saved.Question is not saved.
"""

exc_prompt_2="""
Main flow: 
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 3: Admin presses one Lesson on the Lesson list.
Step 4: The system redirects Admin to the "Lesson Information" page of selected lesson which show the information of that Lesson and Topic list included in that Lesson.
Step 5: Admin presses one Topic on the Topic list of selected Lesson.
Step 6: The system redirects Admin to the "Topic Information" page of selected topic which show the information of that selected topic and vocabulary word list included in that Topic.
Step 7: Admin presses one word on vocabulary word list.
Step 8: The system redirects Admin to the "Word Information" page of selected word which show the information of that selected word and button "Add new question".
Step 9: Admin presses button "Add new question".
Step 10: The system pop up dialog to choose type of question Admin want to create.
Step 11: Admin presses option add "Create questions for the selection test".
Step 12: The system redirects Admin to the "Add new question for selection test" page of selected word which show all the fields that admin need to input to create a selection test and button 'Save' and 'Cancel'. These complusary fields are: one textfield "sentence" for Admins to input sentence that learner need to fill in blank and the position that the word learner need to fill must present by symbol '...',. One textfield 'correct answer' for Admins to input the correct answer word, three textfield 'other option 1', 'other option 2', 'other option 3' for Admins to input the 3 different wrong answer word.
Step 13: Admin fills in the sentence field by a valid sentence has a symbol '...', a valid sentence must be below 250 characters and must not contains any special character. 
Step 14: Admin fills 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 15: Admin presses button 'Save' to save the question information.
Step 16: System save the question and redirect Admin to the "Word Information" page of selected word.

Exception flow 2: Admin choose cancel the question and confirm the cancel.
At step 13 of the basic flow: Admin presses button 'Cancel'.
Step 14: System show dialog 'Are you sure you want to cancel the question?' with 2 option buttons 'Confirm' and 'Stay'.
Step 15: Admin presses the buttons 'Confirm'.
Step 16: The system redirects Admin to the "Word Information" page of selected word and Question is not saved.
"""

prompt_all = """
Main flow: 
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 3: Admin presses one Lesson on the Lesson list.
Step 4: The system redirects Admin to the "Lesson Information" page of selected lesson which show the information of that Lesson and Topic list included in that Lesson.
Step 5: Admin presses one Topic on the Topic list of selected Lesson.
Step 6: The system redirects Admin to the "Topic Information" page of selected topic which show the information of that selected topic and vocabulary word list included in that Topic.
Step 7:Admin presses one word on vocabulary word list.
Step 8:The system redirects Admin to the "Word Information" page of selected word which show the information of that selected word and button "Add new question".
Step 9:Admin presses button "Add new question".
Step 10:The system pop up dialog to choose type of question Admin want to create.
Step 11: Admin presses option add "Create questions for the selection test".
Step 12: The system redirects Admin to the "Add new question for selection test" page of selected word which show all the fields that admin need to input to create a selection test and button 'Save' and 'Cancel'. These compulsory fields are: text field "sentence" for inputting sentence that learner need to fill in blank and the position that the word learner need to fill must present by symbol '...'and "sentence" must not be skip,  a valid sentence must be below 250 characters and not contains any special character. one text field 'correct answer' for Admins to input the correct answer word, three text field 'other option 1', 'other option 2', 'other option 3' for Admins to input the 3 different wrong answer word.
Step 13: Admin fills the field "sentence" with a valid sentence has a symbol '...', 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 14: Admin presses button 'Save' to save the question information.
Step 15: System save the question and redirect Admin to the "Word Information" page of selected word.

Alternative flow: 
Alternative flow 1: Admin go to "Lesson Management" page from Homepage.
At step 1 of the basic flow: Admin presses the "Lesson Management" tab at Homepage.
Go back to step 2 in the basic flow and continue with the steps from step 2.

Alternative flow 2: Admin skip compulsory field "sentence".
At step 13 of the basic flow: Admin field all the compulsory field except field "sentence".
Step 14: Admin presses button 'Save'.
Step 15: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing "sentence" field.
Step 16: Admin re-fill the missing field.
Step 17: Admin presses button 'Save'.

Alternative flow 3: Admin skip compulsory field "'correct answer'.
At step 13 of the basic flow: Admin field all the compulsory field except field 'correct answer'.
Step 14: Admin presses button 'Save'.
Step 15: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'correct answer' field.
Step 16: Admin re-fill the missing field.
Step 17: Admin presses button 'Save'.

Alternative flow 4: Admin skip compulsory field 'other option 1'.
At step 13 of the basic flow: Admin field all the compulsory field except field 'other option 1'.
Step 14: Admin presses button 'Save'.
Step 15: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'other option 1' field.
Step 16: Admin re-fill the missing field.
Step 17: Admin presses button 'Save'.

Alternative flow 5: Admin skip compulsory field 'other option 2'.
At step 13 of the basic flow: Admin field all the compulsory field except field 'other option 2'.
Step 14: Admin presses button 'Save'.
Step 15: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'other option 2' field.
Step 16: Admin re-fill the missing field.
Step 17: Admin presses button 'Save'.

Alternative flow 6: Admin skip compulsory field 'other option 3'.
At step 13 of the basic flow: Admin field all the compulsory field except field 'other option 3'.
Step 14: Admin presses button 'Save'.
Step 15: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'other option 3'field.
Step 16: Admin re-fill the missing field.
Step 17: Admin presses button 'Save'.

Alternative flow 7: Admin choose cancel the question and choose to stay.
At step 13 of the basic flow: Admin presses button 'Cancel'.
Step 14: System show dialog 'Are you sure you want to cancel the question?' with 2 option buttons 'Confirm' and 'Stay'.
Step 15: Admin presses the buttons 'Stay'.
Step 16: The dialog closed and the work on that question was kept.
Step 17: Admin presses button 'Save'.

Alternative flow 8: Admin enters invalid sentences.
At step 13 of the basic flow: Admin fills all the other compulsory field but fills in the 'sentence' field by an invalid sentence.
Step 14: System show notification that 'Sentence is invalid', prompt Admin to re-enter the "sentence" field.
Step 16: Admin re-enter the missing field.
Step 17: Admin presses button 'Save'.

Exception flow: 
Exception flow 1: Admin leave by button back on browser without saving the question.
At step 13 of the basic flow: Admin presses the button back on the browser. 
Step 14: The system redirects Admin to the "Word Information" page of selected word and Question is not saved. Question is not saved.

Exception flow 2: Admin choose cancel the question and confirm the cancel.
At step 13 of the basic flow: Admin presses button 'Cancel'.
Step 14: System show dialog 'Are you sure you want to cancel the question?' with 2 option buttons 'Confirm' and 'Stay'.
Step 15: Admin presses the buttons 'Confirm'.
Step 16: The system redirects Admin to the "Word Information" page of selected word and Question is not saved.
"""

In [5]:
promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

promptSubScenario1 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_1}
  ]
sub1_gpt_response = ask(promptSubScenario1, client, model)

promptSubScenario2 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_2}
  ]
sub2_gpt_response = ask(promptSubScenario2, client, model)

promptSubScenario3 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_3}
  ]
sub3_gpt_response = ask(promptSubScenario3, client, model)

promptSubScenario4 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_4}
  ]
sub4_gpt_response = ask(promptSubScenario4, client, model)

promptSubScenario5 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_5}
  ]
sub5_gpt_response = ask(promptSubScenario5, client, model)

promptSubScenario6 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_6}
  ]
sub6_gpt_response = ask(promptSubScenario6, client, model)

promptSubScenario7 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_7}
  ]
sub7_gpt_response = ask(promptSubScenario7, client, model)

promptSubScenario8 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_8}
  ]
sub8_gpt_response = ask(promptSubScenario8, client, model)

promptSubScenario9 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": exc_prompt_1}
  ]
sub9_gpt_response = ask(promptSubScenario9, client, model)

promptSubScenario10 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": exc_prompt_2}
  ]
sub10_gpt_response = ask(promptSubScenario10, client, model)


filtercontent = main_gpt_response + "\n" + sub1_gpt_response + "\n" + sub2_gpt_response + "\n" + sub3_gpt_response  + "\n" + sub4_gpt_response + "\n" + sub5_gpt_response + "\n" + sub6_gpt_response + "\n" + sub7_gpt_response + "\n" + sub8_gpt_response + "\n" + sub9_gpt_response + "\n" + sub10_gpt_response


promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '/n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)


filtercontent += "\n" + condition_scenario_response
print(filtercontent)

promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print(filter_gpt_response)
write_string_to_file(f"D:\\GPT-testing\\ResultSet\\5.15\\{project_name}\\{usecase_name}-1.txt", filter_gpt_response)

{'Sentence field': {'condition': {'valid': "contains '...', below 250 characters, does not contain any special character", 'invalid': 'skipped, above 250 characters, contains special characters'}, 'type': 'text field'}, 'Correct answer field': {'condition': {'valid': 'filled', 'invalid': 'skipped'}, 'type': 'text field'}, 'Other option 1 field': {'condition': {'valid': 'filled', 'invalid': 'skipped'}, 'type': 'text field'}, 'Other option 2 field': {'condition': {'valid': 'filled', 'invalid': 'skipped'}, 'type': 'text field'}, 'Other option 3 field': {'condition': {'valid': 'filled', 'invalid': 'skipped'}, 'type': 'text field'}}
1. Scenario: Sentence field skipped
2. Scenario: Sentence field above 250 characters
3. Scenario: Sentence field contains special characters
4. Scenario: Correct answer field skipped
5. Scenario: Other option 1 field skipped
6. Scenario: Other option 2 field skipped
7. Scenario: Other option 3 field skipped
1. Successful Creation of a Selection Test Question
2. 

In [6]:
print(filter_gpt_response)

- Successful Creation of a Selection Test Question
- Creation of a Selection Test Question with Invalid Sentence (Exceeds Character Limit or Contains Special Characters)
- Creation of a Selection Test Question with Duplicate Wrong Answers
- Creation of a Selection Test Question with Incorrect Word Form as Correct Answer
- Cancellation of Selection Test Question Creation
- Admin Accesses Lesson Management Directly from Homepage
- Admin skips the "sentence" field while creating a new question for the selection test
- Admin skips the 'correct answer' field while creating a new question for the selection test
- Admin skips compulsory field 'other option 1' during question creation
- Admin skips 'other option 2' field during question creation
- Admin skips 'other option 3' field during question creation
- Admin Decides to Cancel but Then Chooses to Stay and Continue Editing
- Admin navigates back using browser back button before saving the question
- Admin Cancels Question Creation and Conf

In [7]:
promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

promptSubScenario1 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_1}
  ]
sub1_gpt_response = ask(promptSubScenario1, client, model)

promptSubScenario2 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_2}
  ]
sub2_gpt_response = ask(promptSubScenario2, client, model)

promptSubScenario3 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_3}
  ]
sub3_gpt_response = ask(promptSubScenario3, client, model)

promptSubScenario4 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_4}
  ]
sub4_gpt_response = ask(promptSubScenario4, client, model)

promptSubScenario5 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_5}
  ]
sub5_gpt_response = ask(promptSubScenario5, client, model)

promptSubScenario6 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_6}
  ]
sub6_gpt_response = ask(promptSubScenario6, client, model)

promptSubScenario7 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_7}
  ]
sub7_gpt_response = ask(promptSubScenario7, client, model)

promptSubScenario8 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_8}
  ]
sub8_gpt_response = ask(promptSubScenario8, client, model)

promptSubScenario9 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": exc_prompt_1}
  ]
sub9_gpt_response = ask(promptSubScenario9, client, model)

promptSubScenario10 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": exc_prompt_2}
  ]
sub10_gpt_response = ask(promptSubScenario10, client, model)


filtercontent = main_gpt_response + "\n" + sub1_gpt_response + "\n" + sub2_gpt_response + "\n" + sub3_gpt_response  + "\n" + sub4_gpt_response + "\n" + sub5_gpt_response + "\n" + sub6_gpt_response + "\n" + sub7_gpt_response + "\n" + sub8_gpt_response + "\n" + sub9_gpt_response + "\n" + sub10_gpt_response


promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '/n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)


filtercontent += "\n" + condition_scenario_response
print(filtercontent)

promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print(filter_gpt_response)
write_string_to_file(f"D:\\GPT-testing\\ResultSet\\5.15\\{project_name}\\{usecase_name}-2.txt", filter_gpt_response)

{'Sentence field': {'condition': {'valid': "contains '...', below 250 characters, does not contain any special character, must not be skipped", 'invalid': "skipped, above 250 characters, contains special characters, does not contain '...'"}, 'type': 'text field'}, 'Correct answer field': {'condition': {'valid': 'must not be skipped', 'invalid': 'skipped'}, 'type': 'text field'}, 'Other option 1 field': {'condition': {'valid': 'must not be skipped', 'invalid': 'skipped'}, 'type': 'text field'}, 'Other option 2 field': {'condition': {'valid': 'must not be skipped', 'invalid': 'skipped'}, 'type': 'text field'}, 'Other option 3 field': {'condition': {'valid': 'must not be skipped', 'invalid': 'skipped'}, 'type': 'text field'}}
1. Scenario: Sentence field skipped
2. Scenario: Sentence field above 250 characters
3. Scenario: Sentence field contains special characters
4. Scenario: Sentence field does not contain '...'
5. Scenario: Correct answer field skipped
6. Scenario: Other option 1 field

In [8]:
promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

promptSubScenario1 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_1}
  ]
sub1_gpt_response = ask(promptSubScenario1, client, model)

promptSubScenario2 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_2}
  ]
sub2_gpt_response = ask(promptSubScenario2, client, model)

promptSubScenario3 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_3}
  ]
sub3_gpt_response = ask(promptSubScenario3, client, model)

promptSubScenario4 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_4}
  ]
sub4_gpt_response = ask(promptSubScenario4, client, model)

promptSubScenario5 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_5}
  ]
sub5_gpt_response = ask(promptSubScenario5, client, model)

promptSubScenario6 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_6}
  ]
sub6_gpt_response = ask(promptSubScenario6, client, model)

promptSubScenario7 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_7}
  ]
sub7_gpt_response = ask(promptSubScenario7, client, model)

promptSubScenario8 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_8}
  ]
sub8_gpt_response = ask(promptSubScenario8, client, model)

promptSubScenario9 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": exc_prompt_1}
  ]
sub9_gpt_response = ask(promptSubScenario9, client, model)

promptSubScenario10 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": exc_prompt_2}
  ]
sub10_gpt_response = ask(promptSubScenario10, client, model)


filtercontent = main_gpt_response + "\n" + sub1_gpt_response + "\n" + sub2_gpt_response + "\n" + sub3_gpt_response  + "\n" + sub4_gpt_response + "\n" + sub5_gpt_response + "\n" + sub6_gpt_response + "\n" + sub7_gpt_response + "\n" + sub8_gpt_response + "\n" + sub9_gpt_response + "\n" + sub10_gpt_response


promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '/n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)


filtercontent += "\n" + condition_scenario_response
print(filtercontent)

promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print(filter_gpt_response)
write_string_to_file(f"D:\\GPT-testing\\ResultSet\\5.15\\{project_name}\\{usecase_name}-3.txt", filter_gpt_response)

{'Sentence field': {'condition': {'valid': "contains '...', below 250 characters, does not contain any special character", 'invalid': 'skipped, above 250 characters, contains special characters'}, 'type': 'text field'}, 'Correct answer field': {'condition': {'valid': 'filled', 'invalid': 'skipped'}, 'type': 'text field'}, 'Other option 1 field': {'condition': {'valid': 'filled', 'invalid': 'skipped'}, 'type': 'text field'}, 'Other option 2 field': {'condition': {'valid': 'filled', 'invalid': 'skipped'}, 'type': 'text field'}, 'Other option 3 field': {'condition': {'valid': 'filled', 'invalid': 'skipped'}, 'type': 'text field'}}
1. Scenario: Sentence field skipped
2. Scenario: Sentence field above 250 characters
3. Scenario: Sentence field contains special characters
4. Scenario: Correct answer field skipped
5. Scenario: Other option 1 field skipped
6. Scenario: Other option 2 field skipped
7. Scenario: Other option 3 field skipped
Based on the provided main flow of a use case, here are

In [9]:
promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

promptSubScenario1 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_1}
  ]
sub1_gpt_response = ask(promptSubScenario1, client, model)

promptSubScenario2 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_2}
  ]
sub2_gpt_response = ask(promptSubScenario2, client, model)

promptSubScenario3 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_3}
  ]
sub3_gpt_response = ask(promptSubScenario3, client, model)

promptSubScenario4 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_4}
  ]
sub4_gpt_response = ask(promptSubScenario4, client, model)

promptSubScenario5 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_5}
  ]
sub5_gpt_response = ask(promptSubScenario5, client, model)

promptSubScenario6 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_6}
  ]
sub6_gpt_response = ask(promptSubScenario6, client, model)

promptSubScenario7 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_7}
  ]
sub7_gpt_response = ask(promptSubScenario7, client, model)

promptSubScenario8 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_8}
  ]
sub8_gpt_response = ask(promptSubScenario8, client, model)

promptSubScenario9 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": exc_prompt_1}
  ]
sub9_gpt_response = ask(promptSubScenario9, client, model)

promptSubScenario10 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": exc_prompt_2}
  ]
sub10_gpt_response = ask(promptSubScenario10, client, model)


filtercontent = main_gpt_response + "\n" + sub1_gpt_response + "\n" + sub2_gpt_response + "\n" + sub3_gpt_response  + "\n" + sub4_gpt_response + "\n" + sub5_gpt_response + "\n" + sub6_gpt_response + "\n" + sub7_gpt_response + "\n" + sub8_gpt_response + "\n" + sub9_gpt_response + "\n" + sub10_gpt_response


promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '/n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)


filtercontent += "\n" + condition_scenario_response
print(filtercontent)

promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print(filter_gpt_response)
write_string_to_file(f"D:\\GPT-testing\\ResultSet\\5.15\\{project_name}\\{usecase_name}-4.txt", filter_gpt_response)

{'Sentence field': {'condition': {'valid': "contains '...', below 250 characters, does not contain any special character", 'invalid': 'skipped, above 250 characters, contains special characters'}, 'type': 'text field'}, 'Correct answer field': {'condition': {'valid': 'filled', 'invalid': 'skipped'}, 'type': 'text field'}, 'Other option 1 field': {'condition': {'valid': 'filled', 'invalid': 'skipped'}, 'type': 'text field'}, 'Other option 2 field': {'condition': {'valid': 'filled', 'invalid': 'skipped'}, 'type': 'text field'}, 'Other option 3 field': {'condition': {'valid': 'filled', 'invalid': 'skipped'}, 'type': 'text field'}}
1. Scenario_SentenceField_Skipped
2. Scenario_SentenceField_Above250Characters
3. Scenario_SentenceField_ContainsSpecialCharacters
4. Scenario_CorrectAnswerField_Skipped
5. Scenario_OtherOption1Field_Skipped
6. Scenario_OtherOption2Field_Skipped
7. Scenario_OtherOption3Field_Skipped
1. Successful Creation of a Selection Test Question
2. Creation of a Selection T

In [10]:
promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

promptSubScenario1 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_1}
  ]
sub1_gpt_response = ask(promptSubScenario1, client, model)

promptSubScenario2 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_2}
  ]
sub2_gpt_response = ask(promptSubScenario2, client, model)

promptSubScenario3 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_3}
  ]
sub3_gpt_response = ask(promptSubScenario3, client, model)

promptSubScenario4 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_4}
  ]
sub4_gpt_response = ask(promptSubScenario4, client, model)

promptSubScenario5 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_5}
  ]
sub5_gpt_response = ask(promptSubScenario5, client, model)

promptSubScenario6 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_6}
  ]
sub6_gpt_response = ask(promptSubScenario6, client, model)

promptSubScenario7 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_7}
  ]
sub7_gpt_response = ask(promptSubScenario7, client, model)

promptSubScenario8 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_8}
  ]
sub8_gpt_response = ask(promptSubScenario8, client, model)

promptSubScenario9 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": exc_prompt_1}
  ]
sub9_gpt_response = ask(promptSubScenario9, client, model)

promptSubScenario10 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": exc_prompt_2}
  ]
sub10_gpt_response = ask(promptSubScenario10, client, model)


filtercontent = main_gpt_response + "\n" + sub1_gpt_response + "\n" + sub2_gpt_response + "\n" + sub3_gpt_response  + "\n" + sub4_gpt_response + "\n" + sub5_gpt_response + "\n" + sub6_gpt_response + "\n" + sub7_gpt_response + "\n" + sub8_gpt_response + "\n" + sub9_gpt_response + "\n" + sub10_gpt_response


promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '/n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)


filtercontent += "\n" + condition_scenario_response
print(filtercontent)

promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print(filter_gpt_response)
write_string_to_file(f"D:\\GPT-testing\\ResultSet\\5.15\\{project_name}\\{usecase_name}-5.txt", filter_gpt_response)

{'Sentence field': {'condition': {'valid': "contains '...', below 250 characters, does not contain any special character", 'invalid': 'skipped, over 250 characters, contains special characters'}, 'type': 'text field'}, 'Correct answer field': {'condition': {'valid': 'must be entered', 'invalid': 'skipped'}, 'type': 'text field'}, 'Other option 1 field': {'condition': {'valid': 'must be entered', 'invalid': 'skipped'}, 'type': 'text field'}, 'Other option 2 field': {'condition': {'valid': 'must be entered', 'invalid': 'skipped'}, 'type': 'text field'}, 'Other option 3 field': {'condition': {'valid': 'must be entered', 'invalid': 'skipped'}, 'type': 'text field'}}
1. Scenario: Sentence field skipped
2. Scenario: Sentence field over 250 characters
3. Scenario: Sentence field contains special characters
4. Scenario: Correct answer field skipped
5. Scenario: Other option 1 field skipped
6. Scenario: Other option 2 field skipped
7. Scenario: Other option 3 field skipped
1. Successful Creatio